# XY16-N - interrogation time - XY16-N

In [1]:
from collections import OrderedDict
import datetime
import numpy as np
import time
import os

try: 
    pulsedmasterlogic
except NameError:
    manager.startModule('logic', 'pulsedmasterlogic')
try: 
    scannerlogic
except NameError:
    manager.startModule('logic', 'scannerlogic')
try: 
    optimizer
except NameError:
    manager.startModule('logic', 'optimizer')
try:
    sequencegenerator
except NameError:
    manager.startModule('logic', 'sequencegenerator')
try:
    pulsedmeasurementlogic
except NameError:
    manager.startModule('gui', 'pulsedmeasurementlogic')   
try:
    poimanager
except NameError:
    manager.startModule('logic', 'poimaganer')    
    
def write_to_logfile(nametag, timestamp, name, **kwargs):
    """ Write parameters to custom logfile with name nametag """
    if type(timestamp) is not str:
        timestamp = str(timestamp)
    parameters = list(kwargs)
    if len(parameters) == 1 and type(kwargs[parameters[0]]) is OrderedDict:
        param_dict = kwargs[parameters[0]]
        parameters = list(param_dict)
        kwargs = param_dict
    log_dir = pulsedmeasurementlogic.savelogic().get_path_for_module('CustomLogfiles')
    log_path = os.path.join(log_dir, nametag + '.txt')
    if not os.path.isfile(log_path):
        with open(log_path, 'w') as logfile:
            logfile.write('# timestamp\t')
            logfile.write('# Name\t')
            for param in parameters:
                logfile.write(param + '\t')
            logfile.write('\n#\n')
    with open(log_path, 'a') as logfile:
        logfile.write(timestamp + '\t')
        logfile.write(name + '\t')
        for param in parameters:
            logfile.write('{0:3.6e}\t'.format(kwargs[param]))
        logfile.write('\n')
    return

def refocus_poi(poi=None, max_drift=0.5, min_counts=10e+3):
    if poi==None:
        return False
    scannerlogic.stop_scanning()
    poimanager.go_to_poi(poi)
    pos_start = scannerlogic.get_position()
    NV_name =poimanager.poi_list[poi]._name
    poimanager.optimise_poi(poi)
    while optimizer.module_state() == 'idle':
        time.sleep(0.2)
    while optimizer.module_state() != 'idle':
        time.sleep(0.2)
    time.sleep(5)
    pos_end=scannerlogic.get_position()
    print(counterlogic.countdata.mean(),abs(pos_end[0]-pos_start[0]))
    if (abs(pos_end[0]-pos_start[0])>max_drift or abs(pos_end[1]-pos_start[1])>max_drift or 
       abs(pos_end[2]-pos_start[2])>max_drift or counterlogic.countdata.mean()<min_counts):
        scannerlogic.set_position('',x=pos_start[0],y=pos_start[1],z=pos_start[2])
        poimanager.set_new_position(poi,pos_start)
        print(abs(pos_end[0]-pos_start[0])>max_drift or abs(pos_end[1]-pos_start[1])>max_drift or 
       abs(pos_end[2]-pos_start[2])>max_drift or counterlogic.countdata.mean()<min_counts)
        return False
        write_to_logfile('PositionRefocus_log', str(datetime.datetime.now()),name= NV_name, 
                     x_pos=pos_start[0], y_pos=pos_start[1], z_pos=pos_start[2],sucess=0)
    write_to_logfile('PositionRefocus_log', str(datetime.datetime.now()),name= NV_name, 
                     x_pos=pos_end[0], y_pos=pos_end[1], z_pos=pos_end[2],sucess=1)
    return True

def do_xy16(m_time, save_tag='', stash_raw_data_tag = ''):
    
    pulsedmeasurementlogic.measurement_tag = save_tag
    
    # If everything is properly set, we can start a measurement simply by calling:
    pulsedmasterlogic.toggle_pulsed_measurement(True, stash_raw_data_tag)
    # Wait until the pulsedmeasurementlogic is actually busy and the measurement is running
    while pulsedmeasurementlogic.module_state() != 'locked':
        time.sleep(0.2)
        
    user_terminated = False
    start_time = time.time()
    while time.time() - start_time < m_time:
        if pulsedmeasurementlogic.module_state()=='idle':
            user_terminated = True
            time.sleep(0.2)    
            break
        time.sleep(0.5)
        
    pulsedmasterlogic.toggle_pulsed_measurement(False, stash_raw_data_tag)
    # Wait until the pulsedmeasurementlogic is actually idle and the measurement is stopped
    while pulsedmeasurementlogic.module_state() == 'locked':
        time.sleep(0.2)
    
    if save_tag=='':
        pulsedmasterlogic.save_measurement_data(tag='XY16-'+str(generate_params['xy16_order'])+'_'+NV_name, with_error=True)
    time.sleep(2)
    return user_terminated

def do_xy16_refocus(poi, measurement_time, refocus_interval):
    pulsedmeasurementlogic._saved_raw_data.clear()
    end_measure=False
    xy16_total_time = measurement_time
    xy16_runtime = 0.0
    measurement_time = refocus_interval
    end_measure = do_xy16(refocus_interval, 'xy16_refocus', 'xy16_refocus')
    xy16_runtime += refocus_interval
    while xy16_total_time > xy16_runtime:
        for i in range(1):
            end_measure = not refocus_poi(poi)
            if not end_measure:
                break
        if end_measure:
            break
        end_measure = do_xy16(refocus_interval, 'xy16_refocus', 'xy16_refocus')
        xy16_runtime += refocus_interval
        if end_measure:
            break
    write_to_logfile('xy16_log', str(datetime.datetime.now()),name= NV_name,runtime=xy16_runtime)
    time.sleep(2)
    return end_measure

In [2]:
# Get the dictionary containing the default parameters for the desired sequence and alter the values as needed.
# generate_params = pulsedmasterlogic.generate_method_params['rabi']
generate_params=OrderedDict()
generate_params['name'] = 'Corr_XY16_long'
generate_params['tau_inte'] = 278.0e-9
generate_params['tau_start'] = 2000.0e-9
generate_params['tau_step'] = 50.0e-9
generate_params['xy16_order'] = 5
generate_params['num_of_points'] = 80
generate_params['interval'] = 3.0e-6
generate_params['num_of_intervals'] = 2
generate_params['alternating'] = True

tau_array = generate_params['tau_start'] + np.arange(generate_params['num_of_points'])*generate_params['tau_step']

for j in range(generate_params['num_of_intervals']):
    a = tau_array[-1] + generate_params['interval'] + np.arange(generate_params['num_of_points'])*generate_params['tau_step']
    tau_array = np.append(tau_array, a)

# Call the actual generation method using a wrapper method and pass the parameter set
pulsedmasterlogic.generate_predefined_sequence('Corr_XY16_long', generate_params)
# Give it a moment to generate
time.sleep(0.5)

# The created object is a PulseBlockEnsemble instance, a recipe to create a waveform.
# You can get all created PulseBlockEnsembles using this property (keys are ensemble names, values are the object instances):
#for ensemble_name in pulsedmasterlogic.saved_pulse_block_ensembles.keys():
#    print(ensemble_name)

# The "with_load" flag can be used to also load the waveform into the channels after creating it
pulsedmasterlogic.sample_sequence('Corr_XY16_long', with_load=True)
# Wait for the sampling to finish. There is a status dictionary that can be used to check for process status.
while pulsedmasterlogic.status_dict['sampload_busy']:
    time.sleep(0.1)
    
pulsedmasterlogic.set_measurement_settings(invoke_settings=False, 
                                           controlled_variable = tau_array,
                                           number_of_lasers=2*len(tau_array), 
                                           laser_ignore_list=[], 
                                           alternating=True, 
                                           units=('s', 'arb. u.'))
time.sleep(0.5)

pulsedmasterlogic.set_fast_counter_settings(record_length=4.0e-6, number_of_gates=generate_params['num_of_points'])
time.sleep(0.5)

import winsound
frequency = 300  # Set Frequency To 2500 Hertz
duration = 1000  # Set Duration To 1000 ms == 1 second
winsound.Beep(frequency, duration)
# You can get a read-only view of the current measurement settings by using this property:
# for setting, value in pulsedmasterlogic.measurement_settings.items():
#     print('{0}:\n  {1}'.format(setting, value))

In [3]:
poi=poimanager.active_poi.get_key()
NV_name =poimanager.poi_list[poi]._name

In [4]:
refocus_poi(poi)

142031.66666666666 6.156114921455002e-08


True

In [5]:
pulsedmasterlogic.set_timer_interval(3)

In [6]:
measurement_time = 60*60*12
refocus_interval = 2*60

In [7]:
do_xy16_refocus(poi, measurement_time, refocus_interval)

142031.66666666666 6.107218703206628e-08
142031.66666666666 3.969745259449207e-08
142031.66666666666 3.3462614856061355e-08
142031.66666666666 2.9298055836443463e-08
142031.66666666666 2.070697669661977e-08
2D gaussian fit not successfull
142031.66666666666 0.0
2D gaussian fit not successfull
142031.66666666666 0.0
142031.66666666666 4.058688775029803e-08
142031.66666666666 1.629829370551641e-08
142031.66666666666 1.0755654792433451e-08
142031.66666666666 6.3344140415054105e-09
142031.66666666666 5.436413075146049e-09
142031.66666666666 1.820496455367653e-08
142031.66666666666 7.70515225511785e-09
142031.66666666666 9.094027114127612e-09
142031.66666666666 1.761640128433725e-08
142031.66666666666 8.542308427501502e-09
142031.66666666666 1.0751996217634e-08
142031.66666666666 2.109973246992115e-08
142031.66666666666 2.3322832343082307e-09
142031.66666666666 1.099881241204247e-08
142031.66666666666 8.918514158504494e-09
142031.66666666666 9.435962640465408e-09
142031.66666666666 4.891690

Traceback (most recent call last):
  File "C:\Users\pi3\Documents\GitHub\qudi\logic\pulsed\pulsed_measurement_logic.py", line 955, in do_fit
    x_fit, y_fit, result = self.fc.do_fit(data[0], data[1])
  File "C:\Users\pi3\Documents\GitHub\qudi\logic\fit_logic.py", line 409, in do_fit
    **kwargs)
  File "C:\Users\pi3\Documents\GitHub\qudi\logic\fitmethods\sinemethods.py", line 700, in make_sineexponentialdecay_fit
    error, params = estimator(x_axis, data, params)
  File "C:\Users\pi3\Documents\GitHub\qudi\logic\fitmethods\sinemethods.py", line 818, in estimate_sineexponentialdecay
    iter_steps = int(1/(frequency_max*min_x_diff))
OverflowError: cannot convert float infinity to integer
Traceback (most recent call last):
  File "C:\Users\pi3\Documents\GitHub\qudi\logic\pulsed\pulsed_measurement_logic.py", line 955, in do_fit
    x_fit, y_fit, result = self.fc.do_fit(data[0], data[1])
  File "C:\Users\pi3\Documents\GitHub\qudi\logic\fit_logic.py", line 409, in do_fit
    **kwargs)
  F

False

In [8]:
# save measurement
pulsedmasterlogic.save_measurement_data(tag='Corr_XY16-'+str(generate_params['xy16_order'])+'_'+NV_name, with_error=True)